# Dyes example

This follows the BUGS Dyes example as in the [Nimble documentation](https://nature.berkeley.edu/~pdevalpine/nimbleExamples/build_a_model/nimble_build_a_model.html).

In [1]:
import numpy as np
import tensorflow as tf
import edward as ed
from edward.models import Empirical, Normal, Uniform
from scipy.stats import norm

## Model

Edward requires us to specify data shape first.

In [2]:
BATCHES = 5
SAMPLES = 6

Next we specify a probabilistic program.
Note that the rows must be properly ordered (in contrast to BUGS).

In [3]:
sigma_within = Uniform(0.0, 100.0)
sigma_between = Uniform(0.0, 100.0)
theta = Normal(loc=0.0, scale=1e5)
mu = Normal(loc=theta * tf.ones(BATCHES), scale=sigma_between)
y = Normal(loc=tf.reshape(mu, (BATCHES, 1)),
           scale=tf.tile(tf.reshape(sigma_within, (1, 1)), (1, SAMPLES)))

In [11]:
data = tf.transpose(tf.constant([
    [1545, 1540, 1595, 1445, 1595],
    [1520, 1440, 1555, 1550, 1440],
    [1630, 1455, 1440, 1490, 1605],
    [1595, 1515, 1450, 1520, 1560],
    [1510, 1465, 1635, 1480, 1580],
    [1495, 1560, 1545, 1625, 1445],
], dtype = tf.float32))

## Inference

In [12]:
T = 100  # Number of Monte Carlo samples.
inference = ed.HMC(
    latent_vars={
        sigma_within: Empirical(params=tf.zeros(T)),
        sigma_between: Empirical(params=tf.zeros(T)),
        # theta: Empirical(params=tf.zeros(T)),
        # mu: Empirical(params=tf.zeros([T, BATCHES])),
    },
    data={y: data})

In [10]:
inference.run()

ValueError: cannot add op with name inference_139746697271440/1/Normal/loc as that name is already used

In [18]:
import pdb; pdb.set_trace()
inference.run()

--Return--
> <ipython-input-18-a3deee0ab781>(1)<module>()->None
-> import pdb; pdb.set_trace()
(Pdb) c


IndexError: list index out of range

In [19]:
pdb.pm()

> /home/fritz/github/blei-lab/edward/edward/inferences/hmc.py(115)build_update()
-> variable = qz.get_variables()[0]
(Pdb) qz
<ed.RandomVariable 'Empirical_32/' shape=() dtype=float32>
(Pdb) qz.get_variables()
[]
(Pdb) self.latent_vars
{<ed.RandomVariable 'Uniform_2/' shape=() dtype=float32>: <ed.RandomVariable 'Empirical_32/' shape=() dtype=float32>, <ed.RandomVariable 'Uniform_5/' shape=() dtype=float32>: <ed.RandomVariable 'Empirical_35/' shape=() dtype=float32>, <ed.RandomVariable 'Normal_2/' shape=() dtype=float32>: <ed.RandomVariable 'Empirical_38/' shape=() dtype=float32>, <ed.RandomVariable 'Normal_5/' shape=(5,) dtype=float32>: <ed.RandomVariable 'Empirical_41/' shape=(5,) dtype=float32>}
(Pdb) quit


In [16]:
inference.run(theta=1500,
              mu=norm.rvs(loc=1500, scale=50, size=SAMPLES),
              sigma_within=20,
              sigma_between=20)

TypeError: initialize() got an unexpected keyword argument 'mu'

In [78]:
inference.latent_vars

{}